In [1]:
import numpy as np
import cv2
import os
import shutil

### Load model & image path

In [2]:
# load trained model
yolo_model = cv2.dnn.readNetFromDarknet('trained_model/cov_yolov4.cfg','trained_model/cov_yolov4_2000.weights')

# Get all layers from the yolo network
# Loop and find the last layer (output layer) of the yolo network 
yolo_layers = yolo_model.getLayerNames()
yolo_output_layer = [yolo_layers[layer[0] - 1] for layer in yolo_model.getUnconnectedOutLayers()]

In [11]:
# load test image paths
testImges = open('yolo/darknet/sign_data/test.txt', 'r')
testImges = [i for i in testImges.read().split('\n')]

# assign class label
class_labels = ['traffic sign']

#### Copy ground truth files

In [5]:
# get path of annotation test file
imgPath = 'yolo/darknet/sign_data/images/'
txt = [i.split('/')[-1] for i in testImges]
txtFiles = [imgPath+i.split('.')[0]+'.txt' for i in txt]

In [91]:
# Copy ground truth files of test images into mAP-master directory
dest = 'test_model/mAP-master/mAP-master/input/ground-truth/'
for file in txtFiles:
    name = file.split('/')[-1]
    shutil.copyfile(file, dest+name)

#### Copy image files

In [7]:
# get path of image test files
imgPath = 'yolo/darknet/sign_data/images/'
txt = [i.split('/')[-1] for i in testImges]
imgFiles = [imgPath+i.split('.')[0]+'.ppm' for i in txt]

In [8]:
# copy img into map test input directory
dest = 'test_model/mAP-master/mAP-master/input/images-optional/'
for file in imgFiles:
    name = file.split('/')[-1]
    shutil.copyfile(file, dest+name)

#### Get the predicted annotation

In [12]:
def save_anno(imgPath, savePath):
    
    # load test image 
    img =cv2.imread(imgPath)
    imgName = imgPath.split('/')[-1].split('.')[0]    

    # construct a blob for img
    imgBlob = cv2.dnn.blobFromImage(img, 0.003922, (416, 416), swapRB=True, crop=False)

    # get image shape
    height = img.shape[0]
    width = img.shape[1]

    # input preprocessed blob into the model
    yolo_model.setInput(imgBlob)
    # get prediction
    obj_detection_layers = yolo_model.forward(yolo_output_layer)
    
    # create empty lists to save parameters
    class_ids_list = []
    boxes_list = []
    confidences_list = []
    yolo_box = []

    # loop over each of the layer outputs
    for object_detection_layer in obj_detection_layers:
        # loop over the detections
        for object_detection in object_detection_layer:

            # obj_detections[1 to 4] => will have the two center points, box width and box height
            # obj_detections[5] => will have scores for all objects within bounding box
            all_scores = object_detection[5:]
            predicted_class_id = np.argmax(all_scores)
            prediction_confidence = all_scores[predicted_class_id]

            # take only predictions with confidence more than 60%
            if prediction_confidence > 0.60:
                #get the predicted label
                predicted_class_label = class_labels[predicted_class_id]
                #obtain the bounding box co-oridnates for actual image from resized image size
                bounding_box = object_detection[0:4] * np.array([width, height, width, height])
                (box_center_x_pt, box_center_y_pt, box_width, box_height) = bounding_box.astype("int")
                start_x_pt = int(box_center_x_pt - (box_width / 2))
                start_y_pt = int(box_center_y_pt - (box_height / 2))

                # save class id, start x, y, width & height, confidences in a list
                # make sure to pass confidence as float and width and height as integers
                class_ids_list.append(predicted_class_id)
                confidences_list.append(float(prediction_confidence))
                boxes_list.append([start_x_pt, start_y_pt, int(box_width), int(box_height)])
                yolo_box.append([box_center_x_pt, box_center_y_pt, int(box_width), int(box_height)])
                
    # apply non-maxima suppression
    max_value_ids = cv2.dnn.NMSBoxes(boxes_list, confidences_list, 0.5, 0.4)
    
    # create an empty list to store sign annotation inside
    text = []
    
    # loop inside max_value_ids
    for max_valueid in max_value_ids:
        max_class_id = max_valueid[0]
        box = yolo_box[max_class_id]
        centre_x_pt = box[0]/ width
        centre_y_pt = box[1]/ height
        box_width = box[2]/ width
        box_height = box[3]/ height

        #get the predicted class id and label
        predicted_class_id = class_ids_list[max_class_id]
        predicted_class_label = class_labels[predicted_class_id]
        prediction_confidence = confidences_list[max_class_id]

        # save yolo annotation for each sign and store it in a list 
        line = "{} {} {} {} {} {}".format(predicted_class_id, prediction_confidence, centre_x_pt, centre_y_pt, box_width, box_height)
        text.append(line)
            
    # Write file
    filePath = savePath + imgName + '.txt'
    with open(filePath, 'w') as file:
        for item in text:
            file.writelines("%s\n" % item)

In [13]:
basePath = 'yolo/darknet/'
savePath = 'test_model/mAP-master/mAP-master/input/detection-results/'

for img in testImges:
    save_anno(basePath + img, savePath)